In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
from sklearn.metrics import mean_squared_error  # Important!
from sklearn.decomposition import PCA           # Important!
from IPython.display import display

In [5]:
def load_data(filename):
    path = os.path.join("..", "data", "tabular", filename)
    data = pl.read_csv(path).to_numpy()
    X = data[:, :-1].astype(float)
    return X

In [6]:
# Load the dataset
X_train = load_data("train_processed.csv")
